In [1]:
from PIL import Image
from PIL.ImageDraw import Draw
import numpy as np

In [3]:
import tensorflow as tf

In [4]:
frozen_graph_filename = "final_model/frozen_inference_graph.pb"

In [5]:
def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def, name="prefix")
    return graph

In [6]:
graph = load_graph(frozen_graph_filename)

In [7]:
for op in graph.get_operations():
    print(op.name)

prefix/image_tensor
prefix/Cast
prefix/Preprocessor/map/Shape
prefix/Preprocessor/map/strided_slice/stack
prefix/Preprocessor/map/strided_slice/stack_1
prefix/Preprocessor/map/strided_slice/stack_2
prefix/Preprocessor/map/strided_slice
prefix/Preprocessor/map/TensorArray
prefix/Preprocessor/map/TensorArrayUnstack/Shape
prefix/Preprocessor/map/TensorArrayUnstack/strided_slice/stack
prefix/Preprocessor/map/TensorArrayUnstack/strided_slice/stack_1
prefix/Preprocessor/map/TensorArrayUnstack/strided_slice/stack_2
prefix/Preprocessor/map/TensorArrayUnstack/strided_slice
prefix/Preprocessor/map/TensorArrayUnstack/range/start
prefix/Preprocessor/map/TensorArrayUnstack/range/delta
prefix/Preprocessor/map/TensorArrayUnstack/range
prefix/Preprocessor/map/TensorArrayUnstack/TensorArrayScatter/TensorArrayScatterV3
prefix/Preprocessor/map/Const
prefix/Preprocessor/map/TensorArray_1
prefix/Preprocessor/map/TensorArray_2
prefix/Preprocessor/map/while/iteration_counter
prefix/Preprocessor/map/while/Ent

In [8]:
all_tensors = [tensor for op in graph.get_operations() for tensor in op.values()]

In [9]:
all_tensors[0]

<tf.Tensor 'prefix/image_tensor:0' shape=(?, ?, ?, 3) dtype=uint8>

In [10]:
all_tensors[-10:-1]

[<tf.Tensor 'prefix/Postprocessor/Squeeze:0' shape=(?, ?, 4) dtype=float32>,
 <tf.Tensor 'prefix/add/y:0' shape=() dtype=float32>,
 <tf.Tensor 'prefix/add:0' shape=<unknown> dtype=float32>,
 <tf.Tensor 'prefix/detection_boxes:0' shape=<unknown> dtype=float32>,
 <tf.Tensor 'prefix/detection_scores:0' shape=<unknown> dtype=float32>,
 <tf.Tensor 'prefix/detection_multiclass_scores:0' shape=<unknown> dtype=float32>,
 <tf.Tensor 'prefix/detection_classes:0' shape=<unknown> dtype=float32>,
 <tf.Tensor 'prefix/num_detections:0' shape=<unknown> dtype=float32>,
 <tf.Tensor 'prefix/raw_detection_boxes:0' shape=(?, ?, 4) dtype=float32>]

In [11]:
x = graph.get_tensor_by_name('prefix/image_tensor:0')
y1 = graph.get_tensor_by_name('prefix/detection_boxes:0')
y2 = graph.get_tensor_by_name('prefix/detection_scores:0')
y3 = graph.get_tensor_by_name('prefix/detection_classes:0')
y4 = graph.get_tensor_by_name('prefix/detection_multiclass_scores:0')

In [49]:
pilimg = Image.open("../samples/sample3.jpg")
image_array = np.array(pilimg)
image = np.reshape(image_array, (1, image_array.shape[0], image_array.shape[1], image_array.shape[2]))

In [50]:
with tf.Session(graph=graph) as sess:
    results = sess.run([y1, y2, y3, y4], {x: image})

In [51]:
results

[array([[[0.21187793, 0.40324572, 0.47445464, 0.49510583],
         [0.24183397, 0.8225402 , 0.4922157 , 0.9075708 ],
         [0.1892514 , 0.00259001, 0.45484987, 0.05459431],
         [0.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.        ]]], dtype=float32),
 array([[0.9997832 , 0.99580646, 0.99436295, 0.        , 0.        ]],
       dtype=float32),
 array([[3., 3., 3., 1., 1.]], dtype=float32),
 array([[[5.5432320e-06, 5.2750111e-06, 1.2367964e-05, 9.9978322e-01],
         [3.6955178e-03, 1.3113022e-06, 9.2387199e-07, 9.9580646e-01],
         [6.3827038e-03, 8.3327293e-05, 7.4341893e-04, 9.9436295e-01],
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00]]],
       dtype=float32)]

In [52]:
for boxid in range(len(results[0][0])):
    if results[1][0][boxid] > 0.9:
        width = pilimg.size[0]
        height = pilimg.size[1]
        box = results[0][0][boxid]
        xmin = int(box[1]*width)
        ymin = int(box[0]*height)
        xmax = int(box[3]*width)
        ymax = int(box[2]*height)
        predclass = results[2][0][boxid]
        draw = Draw(pilimg)
        draw.rectangle([(xmin, ymin), (xmax, ymax)], outline='red')
        draw.text((xmax+5, ymin+5), "Class: "+str(predclass))

In [47]:
pilimg.show()

In [53]:
#pilimg.save("../samples/sample3_pred.jpg")